In [1]:
import pandas as pd

In [2]:
# import data
articles = pd.read_csv('Work\\McDevitt\\PubMed\\articles.csv')
downloads = pd.read_csv('Work\\McDevitt\\PubMed\\downloads_summary.csv')
humans = downloads[(downloads['STUDY_TYPE']=='human') & (downloads['rejected']==False)]

In [3]:
# original search counts
ori = articles[['K1','K2']].groupby(['K1','K2']).size().reset_index().rename(columns={0:'count'})
ori['count'].astype(int, copy=False)

0       1
1       2
2       1
3       3
4       1
       ..
6042    4
6043    1
6044    1
6045    1
6046    1
Name: count, Length: 6047, dtype: int32

In [4]:
# downloads search counts
dow = articles[articles['PMID'].isin(downloads.PMID)][['PMID', 'TITLE', 'K1', 'K2']]
dow = dow.groupby(['K1','K2']).size().reset_index().rename(columns={0:'count'})
dow['count'].astype(int, copy=False)

0       1
1       3
2       1
3       1
4       1
       ..
4927    1
4928    1
4929    1
4930    3
4931    1
Name: count, Length: 4932, dtype: int32

In [5]:
# human studies count
agg = articles[articles['PMID'].isin(humans.PMID)][['PMID', 'TITLE', 'K1', 'K2']]
agg = agg.groupby(['K1','K2']).size().reset_index().rename(columns={0:'count'})
agg['count'].astype(int, copy=False)

0       1
1       2
2       1
3       1
4       3
       ..
2275    1
2276    1
2277    1
2278    3
2279    1
Name: count, Length: 2280, dtype: int32

In [6]:
# search counts before downloads
ori['count'].sum()

55436

In [7]:
# search counts after downloads
dow['count'].sum()

30543

In [8]:
# get the number of total search counts for human studies
agg['count'].sum()

6833

In [9]:
# fill in missing columns and rows and export
keys = pd.read_excel('Work\\McDevitt\\PubMed\\keywords.xlsx')[['FIRST KEYWORD', 'SECOND KEYWORD']].rename(columns={'FIRST KEYWORD': 'K1', 'SECOND KEYWORD': 'K2'})

ori.pivot_table(values='count', index=ori.K1, columns='K2', fill_value=0, dropna=False).reindex(
    index=keys.K1[~keys.K1.isna()].str.lower(), 
    columns=keys.K2[~keys.K2.isna()].str.lower(),
    fill_value=0
).fillna(0).to_csv('clustered_search_counts.csv')
dow.pivot_table(values='count', index=dow.K1, columns='K2', fill_value=0, dropna=False).reindex(
    index=keys.K1[~keys.K1.isna()].str.lower(), 
    columns=keys.K2[~keys.K2.isna()].str.lower(),
    fill_value=0
).fillna(0).to_csv('clustered_downloads_search_counts.csv')
agg.pivot_table(values='count', index=agg.K1, columns='K2', fill_value=0, dropna=False).reindex(
    index=keys.K1[~keys.K1.isna()].str.lower(), 
    columns=keys.K2[~keys.K2.isna()].str.lower(),
    fill_value=0
).fillna(0).to_csv('clustered_human_search_counts.csv')